In [2]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('not basics').getOrCreate()

avg_df = spark.read.csv("./AvgSalary.csv", header=True)
salary_df = spark.read.csv("./Glassdoor Gender Pay Gap.csv", header=True, inferSchema=True)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/12 02:51:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
salary_df.summary().show()

+-------+-------------------+------+------------------+------------------+---------+--------------+------------------+-----------------+------------------+------------------+
|summary|           JobTitle|Gender|               Age|          PerfEval|Education|          Dept|         Seniority|          BasePay|             Bonus|         CompanyID|
+-------+-------------------+------+------------------+------------------+---------+--------------+------------------+-----------------+------------------+------------------+
|  count|               1000|  1000|              1000|              1000|      989|           999|              1000|             1000|              1000|              1000|
|   mean|               null|  null|            41.393|             3.037|     null|          null|             2.971|        93994.653|          6506.161|           688.204|
| stddev|               null|  null|14.294855504477594|1.4239587980902595|     null|          null|1.3950287112117459|25530.7

In [1]:
salary_df.show()

NameError: name 'salary_df' is not defined

In [6]:
from pyspark.sql import functions as F
df_agg = salary_df.agg(*(F.count(F.when(F.isnull(c),c)).alias(c) for c in salary_df.columns))

df_agg.show()

+--------+------+---+--------+---------+----+---------+-------+-----+---------+
|JobTitle|Gender|Age|PerfEval|Education|Dept|Seniority|BasePay|Bonus|CompanyID|
+--------+------+---+--------+---------+----+---------+-------+-----+---------+
|       0|     0|  0|       0|       11|   1|        0|      0|    0|        0|
+--------+------+---+--------+---------+----+---------+-------+-----+---------+



In [12]:
salary_df.describe(['BasePay', 'Bonus']).show()

+-------+-----------------+------------------+
|summary|          BasePay|             Bonus|
+-------+-----------------+------------------+
|  count|             1000|              1000|
|   mean|        93994.653|          6506.161|
| stddev|25530.79670901769|2479.4433190236978|
|    min|            34208|               149|
|    max|           242105|             50000|
+-------+-----------------+------------------+



In [15]:
salary_df.filter("Gender=='Female'").show()

+-------------------+------+---+--------+-----------+--------------+---------+-------+-----+---------+
|           JobTitle|Gender|Age|PerfEval|  Education|          Dept|Seniority|BasePay|Bonus|CompanyID|
+-------------------+------+---+--------+-----------+--------------+---------+-------+-----+---------+
|            Manager|Female| 60|       4|        PhD|    Management|        3| 140614| 8354|      812|
|            Manager|Female| 62|       4|    Masters|         Sales|        4| 125203| 7808|        5|
|            Manager|Female| 45|       1|High School|   Engineering|        5| 129893| 5120|     1003|
|  Financial Analyst|Female| 47|       5|High School|         Sales|        5| 126190| 8961|      417|
|            Manager|Female| 62|       4|    Masters|Administration|        3| 106008| 6235|      884|
|            Manager|Female| 57|       1|    Masters|    Operations|        3| 145095| 3889|      590|
|            Manager|Female| 45|       2|High School|    Management|     

In [3]:
salary_df = salary_df.na.drop()
salary_df = salary_df.withColumn("TotalSalary", salary_df.BasePay + salary_df.Bonus)
df = salary_df.join(avg_df, salary_df.JobTitle == avg_df.JobTitle, "inner")
df = df.withColumn("BelowAvg", df.TotalSalary < df.AvgSalary)

df_cols = df.columns
# get index of the duplicate columns
duplicate_col_index = list(set([df_cols.index(c) for c in df_cols if df_cols.count(c) == 2]))

# rename by adding suffix '_duplicated'
for i in duplicate_col_index:
    df_cols[i] = df_cols[i] + '_duplicated'

# rename the column in DF
df = df.toDF(*df_cols)

# remove flagged columns
cols_to_remove = [c for c in df_cols if '_duplicated' in c]
df = df.drop(*cols_to_remove)

In [7]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- PerfEval: integer (nullable = true)
 |-- Seniority: integer (nullable = true)
 |-- BelowAvg: boolean (nullable = true)
 |-- JobEncoded: vector (nullable = true)
 |-- EducationEncoded: vector (nullable = true)
 |-- DeptEncoded: vector (nullable = true)



In [5]:
from pyspark.ml.feature import StringIndexer
job_indexer = StringIndexer(inputCol="JobTitle", outputCol="JobIndex")
edu_indexer = StringIndexer(inputCol="Education", outputCol="EducationIndex")
dept_indexer = StringIndexer(inputCol="Dept", outputCol="DeptIndex")
#below_indexer = StringIndexer(inputCol="BelowAvg", outputCol="BelowIndex")
df = job_indexer.fit(df).transform(df)
df = edu_indexer.fit(df).transform(df)
df = dept_indexer.fit(df).transform(df)
#df = below_indexer.fit(df).transform(df)

from pyspark.ml.feature import OneHotEncoder
ohe = OneHotEncoder(inputCols=["JobIndex", "EducationIndex", "DeptIndex"], 
                    outputCols=["JobEncoded", "EducationEncoded", "DeptEncoded"])
df = ohe.fit(df).transform(df)

df = df.drop("Gender", "Education", "Dept", "BasePay", "Bonus", "CompanyID", "TotalSalary", "JobTitle", "AvgSalary", "EducationIndex", "JobIndex", "DeptIndex")
df.show()

In [14]:
df = df.withColumn("BelowAvg",df.BelowAvg.cast('int'))

In [27]:
from pyspark.ml.linalg import Vectors 
from pyspark.ml.feature import VectorAssembler


assembler = VectorAssembler(inputCols=['Age','PerfEval','Seniority','JobEncoded','EducationEncoded','DeptEncoded'], outputCol='features')
output = assembler.transform(df)
final_data = output.select('features','BelowAvg')
final_data.show()

+--------------------+--------+
|            features|BelowAvg|
+--------------------+--------+
|(19,[0,1,2,11,17]...|       0|
|(19,[0,1,2,11,16]...|       0|
|(19,[0,1,2,8,13,1...|       0|
|(19,[0,1,2,11,12,...|       0|
|(19,[0,1,2,11,17]...|       0|
|(19,[0,1,2,4,14,1...|       0|
|(19,[0,1,2,4,12,1...|       0|
|(19,[0,1,2,11,14,...|       0|
|(19,[0,1,2,11,14]...|       0|
|(19,[0,1,2,4,16],...|       0|
|(19,[0,1,2,11,18]...|       0|
|(19,[0,1,2,4,13,1...|       0|
|(19,[0,1,2,11,13,...|       0|
|(19,[0,1,2,4,13,1...|       0|
|(19,[0,1,2,11,13,...|       0|
|(19,[0,1,2,4,13,1...|       0|
|(19,[0,1,2,11,14,...|       0|
|(19,[0,1,2,11,17]...|       0|
|(19,[0,1,2,11],[5...|       0|
|(19,[0,1,2,11,15]...|       0|
+--------------------+--------+
only showing top 20 rows



In [18]:
from pyspark.ml.classification import RandomForestClassifier 
rf = RandomForestClassifier(labelCol='BelowAvg', featuresCol='features')
model = rf.fit(final_data)
model.featureImportances

Exception ignored in: <function JavaWrapper.__del__ at 0x7ff18c5389d0>
Traceback (most recent call last):
  File "/home/ubuntu/spark-3.2.1-bin-hadoop2.7/python/pyspark/ml/wrapper.py", line 39, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'RandomForestClassifier' object has no attribute '_java_obj'


SparseVector(19, {0: 0.5212, 1: 0.0148, 2: 0.3503, 3: 0.0063, 4: 0.0013, 5: 0.01, 6: 0.0031, 7: 0.0045, 8: 0.0038, 9: 0.0028, 10: 0.001, 11: 0.0082, 12: 0.016, 13: 0.0161, 14: 0.0039, 15: 0.0045, 16: 0.0147, 17: 0.0087, 18: 0.0088})

In [19]:
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol='ScaledFeatures')
scaled_final_data = scaler.fit(final_data).transform(final_data)
scaled_final_data.show()

+--------------------+--------+--------------------+
|            features|BelowAvg|      ScaledFeatures|
+--------------------+--------+--------------------+
|(19,[0,1,2,11,17]...|       0|(19,[0,2,11,17],[...|
|(19,[0,1,2,11,16]...|       0|(19,[0,1,2,11,16]...|
|(19,[0,1,2,8,13,1...|       0|(19,[0,1,2,8,13,1...|
|(19,[0,1,2,11,12,...|       0|(19,[0,2,11,12,17...|
|(19,[0,1,2,11,17]...|       0|(19,[0,1,2,11,17]...|
|(19,[0,1,2,4,14,1...|       0|(19,[0,2,4,14,16]...|
|(19,[0,1,2,4,12,1...|       0|(19,[0,1,2,4,12,1...|
|(19,[0,1,2,11,14,...|       0|(19,[0,1,2,11,14,...|
|(19,[0,1,2,11,14]...|       0|(19,[0,1,2,11,14]...|
|(19,[0,1,2,4,16],...|       0|(19,[0,1,2,4,16],...|
|(19,[0,1,2,11,18]...|       0|(19,[0,1,2,11,18]...|
|(19,[0,1,2,4,13,1...|       0|(19,[0,1,2,4,13,1...|
|(19,[0,1,2,11,13,...|       0|(19,[0,2,11,13,18...|
|(19,[0,1,2,4,13,1...|       0|(19,[0,1,2,4,13,1...|
|(19,[0,1,2,11,13,...|       0|(19,[0,1,2,11,13,...|
|(19,[0,1,2,4,13,1...|       0|(19,[0,1,2,4,13

In [35]:
from pyspark.ml.classification import DecisionTreeClassifier 
dt = DecisionTreeClassifier(labelCol='BelowAvg', featuresCol='features', maxDepth=4)
model = dt.fit(final_data)
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_9ea45fc625d0, depth=4, numNodes=27, numClasses=2, numFeatures=19
  If (feature 0 <= 38.5)
   If (feature 2 <= 4.5)
    If (feature 2 <= 3.5)
     Predict: 1.0
    Else (feature 2 > 3.5)
     If (feature 0 <= 33.5)
      Predict: 1.0
     Else (feature 0 > 33.5)
      Predict: 0.0
   Else (feature 2 > 4.5)
    If (feature 0 <= 24.5)
     If (feature 0 <= 18.5)
      Predict: 1.0
     Else (feature 0 > 18.5)
      Predict: 0.0
    Else (feature 0 > 24.5)
     If (feature 3 in {1.0})
      Predict: 1.0
     Else (feature 3 not in {1.0})
      Predict: 0.0
  Else (feature 0 > 38.5)
   If (feature 2 <= 2.5)
    If (feature 0 <= 57.5)
     If (feature 5 in {1.0})
      Predict: 0.0
     Else (feature 5 not in {1.0})
      Predict: 1.0
    Else (feature 0 > 57.5)
     If (feature 3 in {1.0})
      Predict: 1.0
     Else (feature 3 not in {1.0})
      Predict: 0.0
   Else (feature 2 > 2.5)
    If (feature 2 <= 3.5)
     If (feature 0 

In [39]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
pred = model.transform(final_data)
evaluator = MulticlassClassificationEvaluator(labelCol="BelowAvg", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(pred)
print("Accuracy =", accuracy)

Accuracy = 0.8543983822042467
